In [43]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interactive
import lightkurve as lk
from bokeh.models import Select, Row, ColumnDataSource, Slider
from bokeh.plotting import figure, curdoc
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Range1d, Select
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import column
from streamlit import caching
from scipy.signal import savgol_filter
from os.path import dirname, join

#%matplotlib inline

In [60]:
TIC_list = ['172900988', '454140642', ]

In [61]:
def get_data_archive(TIC):
    lcs_QLP = lk.search_lightcurve('TIC ' + str(TIC), author = 'QLP').download_all();

    time_QLP, flux_QLP, flux_QLP_err, flags_QLP = [], [], [], []
    for ii in range(len(lcs_QLP.sector)):
        time_QLP = np.hstack((time_QLP,lcs_QLP[ii].time.value))
        flux_QLP = np.hstack((flux_QLP,lcs_QLP[ii].flux.value))
        flux_QLP_err = np.hstack((flux_QLP_err,lcs_QLP[ii].flux_err.value))
        flags_QLP = np.hstack((flags_QLP,lcs_QLP[ii].quality.value))
    flags_QLP = flags_QLP.astype(bool);

    df = pd.DataFrame(data={
        "time"     : time_QLP,
        "flux"     : flux_QLP,
        "flags"     :  flags_QLP
    })
    return df

In [86]:
def get_data_local_LC(TIC):

    fname_LC = join(os.path.dirname(os.path.realpath('__file__')), 'data/TIC_' + str(TIC) + '_eleanor_LC.csv')
    data_LC = pd.read_csv(fname_LC, header = 0, delimiter=r",", na_filter= True, engine = 'python').values

    df_LC = pd.DataFrame(data={
        "time"     : data_LC[:,0],
        "raw flux"     : data_LC[:,1],
        "corr flux"     :  data_LC[:,2],
        "pca flux"     :  data_LC[:,3],
        "psf flux"     :  data_LC[:,4]
        })

    return df_LC

In [87]:
def get_data_local_LC_diag(TIC):

    fname_LC_diag=join(os.path.dirname(os.path.realpath('__file__')),'data/TIC_'+str(TIC)+'_eleanor_LC_diag.csv')
    data_LC_diag = pd.read_csv(fname_LC_diag, header=0, delimiter=r",", na_filter= True, engine = 'python').values

    df_LC_diag = pd.DataFrame(data={
        "PSF x-com"     : data_LC_diag[:,0],
        "PSF y-com"     : data_LC_diag[:,1],
        "PSF x-width"     :  data_LC_diag[:,2],
        "PSF y-width"     :  data_LC_diag[:,3],
        "PSF rot"     :  data_LC_diag[:,4],
        "PSF bkg"     :  data_LC_diag[:,5],
        })
    return df_LC_diag

In [88]:
output_notebook();

Loading BokehJS ...

In [89]:
def plot_func(TIC):
#    fig = plt.figure(figsize=(20,5))
#    plt.rcParams.update({'font.size': 25})
    df_LC = get_data_local_LC(TIC).copy()
    df_LC_diag = get_data_local_LC_diag(TIC).copy()
    
#    datasource = ColumnDataSource({'x': x, 'y0': y0, 'y1': y1, 'y2': y2})
    
#    distribution = 'Discrete'
#    distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])
    
    time, flux = df_LC['time'].values, df_LC['corr flux'].values
    
    psf_xcom, psf_ycom, psf_xwidth, psf_ywidth, psf_rot, psf_bkg = \
    df_LC_diag['PSF x-com'].values, df_LC_diag['PSF y-com'].values, \
    df_LC_diag['PSF x-width'].values, df_LC_diag['PSF y-width'].values, \
    df_LC_diag['PSF rot'].values, df_LC_diag['PSF bkg'].values
    
    flags = np.zeros(len(time)).astype(bool)
    
    tools='pan,wheel_zoom,box_zoom,xbox_select,reset,save'
    
    p = figure(width=900, height=250, title="TIC " + str(TIC) + ' LC', \
               x_range=(min(time[~flags]),max(time[~flags])), \
               y_range=(0.5,1.2), \
               y_axis_label='Normalized Flux')#,\
#               tools=tools, active_drag="xbox_select")
    p.line(time[~flags], flux[~flags], legend_label="LC", line_width=2, color="black");
    p.circle(time[~flags], flux[~flags], size = 4, color="black");
    p.legend.location = "bottom_right"
    
    p1 = figure(width=900, height=250, title="TIC " + str(TIC) + ' COM', \
                y_range=(0.9,1.1), \
                y_axis_label='PSF COM', x_range=p.x_range)#,\
#                tools=tools, active_drag="xbox_select")
    p1.line(time[~flags], psf_xcom[~flags], legend_label="XCOM", line_width=2, color="red");
    p1.line(time[~flags], psf_ycom[~flags], legend_label="YCOM", line_width=2, color="blue");
    p1.legend.location = "bottom_right"
    
    p2 = figure(width=900, height=250, title="TIC " + str(TIC) + ' PSF Width', \
                y_range=(0.8,1.2), \
                y_axis_label='PSF Width', x_range=p.x_range)#,\
#               tools=tools, x_range=p.x_range)
    p2.line(time[~flags], psf_xwidth[~flags], legend_label="x-width", line_width=2, color="grey");
    p2.line(time[~flags], psf_ywidth[~flags], legend_label="y-width", line_width=2, color="brown");
    p2.legend.location = "bottom_right"  
    
    p3 = figure(width=900, height=250, title="TIC " + str(TIC) + ' PSF Rot + Bkg', \

                y_range=(0.5,1.5), \
                x_axis_label='Time [BJD-2,457,000]', \
                y_axis_label='PSF rot + bkg', x_range=p.x_range)#,\
#                tools=tools, active_drag="xbox_select")
    p3.line(time[~flags], psf_rot[~flags], legend_label="Rotation", line_width=2, color="cyan");
    p3.line(time[~flags], psf_bkg[~flags], legend_label="Background", line_width=2, color="magenta");
    p3.legend.location = "bottom_right"  
    
    show(column(p, p1, p2, p3))
    
    del df_LC, df_LC_diag
    caching.clear_cache()
#
interactive_plot = interactive(plot_func, TIC=TIC_list)#, a=(0,30,0,1)#, f=(0.1, 1))
output = interactive_plot.children[-1]
output.layout.height = '1200px'
interactive_plot

interactive(children=(Dropdown(description='TIC', options=('172900988', '454140642'), value='172900988'), Outp…

In [75]:
#tpf = lk.search_tesscut('TIC ' + str(int(454140642))).download_all(cutout_size=(13,13))

In [ ]:
#tpf[0].interact();

In [ ]:
#tpf[0][500:].interact_sky(magnitude_limit = 21);